In [ ]:
import numpy as np
import WLCgreen as wlc
import propagator 
import special as sp
from numba import jit
import MultiPoint as mp
import imp  # imp.reload(module)

import matplotlib.pyplot as plt
%matplotlib
#%matplotlib inline
#import mpld3
#mpld3.enable_notebook()

In [2]:
def plotlog(x, y, alpha, xrange=1, numx=2):
    xv = np.logspace(np.log10(x), np.log10(x) + xrange, numx)
    C = y / (x**alpha)
    yv = C * xv**(alpha)
    plt.loglog(xv, yv,'k--')

## 2 point

In [3]:
nlam=3
ORDEig=25
d=3
mu=0

nkpts=500
K=np.logspace(-5,7,nkpts)
props=[] # list of propagators at each K
for ii in range(0,nkpts):
    name = ii
    props.append(propagator.propagator(name,K[ii],mu,nlam=nlam))

### AA

In [4]:
fa=0.5
N=[0.1,1.0,10,100,1000,10000,100000]
ana=np.zeros((nkpts,len(N)),dtype=type(1+1j))
dub=np.zeros((nkpts,len(N)),dtype=type(1+1j))
for ii in range(0,nkpts):
    lam=0
    lam0=0
    for nn in range(0,len(N)):
        ana[ii,nn]=mp.IAAresum(N[nn],fa,lam0,lam,props[ii])
        dub[ii,nn]=mp.IAAexplicit(N[nn],fa,lam0,lam,props[ii])

In [6]:
fig, ax = plt.subplots()
for ii in range(0,len(N)):
    col=float(ii)/len(N)
    ax.loglog(K.real*np.sqrt(N[ii]),abs(ana[:,ii])/(N[ii]**2),\
                color=[col, 0.0, 1-col],label='resum '+str(N[ii]))
    #ax.loglog(K.real,abs(ana[:,ii]),\
    #            color=[col, 0.0, 1-col],label='resum '+str(N[ii]))
    ax.loglog(K.real*np.sqrt(N[ii]),abs(dub[:,ii])/(N[ii]**2),'--',\
                color=[col, 0.0, 1-col],label='expl '+str(N[ii]))
    #ax.loglog(K.real,abs(dub[:,ii]),'--',\
    #            color=[col, 0.0, 1-col],label='expl '+str(N[ii]))
ax.legend(bbox_to_anchor=(1.05,1),loc=2)
plotlog(2e4,1e-3,-1) 
ax.set_xlabel('K')
ax.set_title('AA')
ax.set_ylim([10**-8,10**10])
plt.show()

In [ ]:
N=[0.01,0.1,1.0,10,100]
ana=np.zeros((nkpts,len(N)),dtype=type(1+1j))
dub=np.zeros((nkpts,len(N)),dtype=type(1+1j))
fa=0.5
for ii in range(0,nkpts):
    lam=0
    lam0=0
    for nn in range(0,len(N)):
        ana[ii,nn]=mp.IABresum(N[nn],fa,lam0,lam,props[ii])
        dub[ii,nn]=mp.IABexplicit(N[nn],fa,lam0,lam,props[ii])

In [ ]:
fig, ax = plt.subplots()
for ii in range(0,len(N)):
    col=float(ii)/len(N)
    ax.loglog(K.real,abs(ana[:,ii]),\
                color=[col, 0.0, 1-col],label='resum '+str(N[ii]))
    ax.loglog(K.real,abs(dub[:,ii]),'--',\
                color=[col, 0.0, 1-col],label='expl '+str(N[ii]))
ax.legend(bbox_to_anchor=(1.05,1),loc=2)
plotlog(2e4,1e-7,-2)
ax.set_xlabel('K')
ax.set_title('AB')
ax.set_ylim([10**-15,100000])
plt.show()

## 3 Point

In [48]:
nlam=5
ORDEig=3
d=3
mu=0

nkpts=200
K=np.logspace(-4,7,nkpts)
props1=[] # list of propagators at each K
props2=[] # list of propagators at each K
for ii in range(0,nkpts):

    name=ii
    props1.append( propagator.propagator(name,K[ii],mu,nlam=nlam) )

    name=ii+0.5
    props2.append( propagator.propagator(name,K[ii],mu,nlam=nlam) )

#### AAA

In [49]:
def IAAA(N,fa,lam0_1,lam_1,lam0_2,lam_2,p1,p2):
    out=0.0+0.0j
        
    G2, overlapping_other_l2=p2.G_others(lam0_2,lam_2,p1)
    for l1 in range(abs(p1.mu),p1.ORDEig):
        eps1=p1.eig[l1]
        R1=p1.res[l1][lam0_1,lam_1]
        if overlapping_other_l2[l1]==-1: # not overlapping
            out=out+(eps1**-2)*np.exp(eps1*N*fa)*R1*G2[l1]
        else: # overlapping
            l2=overlapping_other_l2[l1]
            eps2=p2.eig[l2]
            R2=p2.res[l2][lam0_2,lam_2]
            
            temp=0.0
            temp=temp + R1*R2*sp.f2(2,eps1,eps2,N*fa)
            temp=temp + (eps1**-2)*np.exp(eps1*N*fa)*R1*p2.a[l2][lam0_2,lam_2]
            temp=temp + (eps2**-2)*np.exp(eps2*N*fa)*R2*p1.a[l1][lam0_1,lam_1]
            out=out+temp
            print('alligned l1,',l1,'l2',l2,'t1',\
                  R1*R2*sp.f2(2,eps1,eps2,N*fa))
            print('alligned l1,',l1,'l2',l2,'t2',\
                  (eps1**-2)*np.exp(eps1*N*fa)*R1*p2.a[l2][lam0_2,lam_2])
            print('alligned l1,',l1,'l2',l2,'t3',\
                  (eps2**-2)*np.exp(eps2*N*fa)*R2*p1.a[l1][lam0_1,lam_1])
        
    G1, overlapping_other_l1=p1.G_others(lam0_1,lam_1,p2)
    for l2 in range(abs(p2.mu),p2.ORDEig):
        if overlapping_other_l1[l2]!=-1:
            continue
        eps2=p2.eig[l2]
        R2=p2.res[l2][lam0_1,lam_1]
        out=out+(eps2**-2)*np.exp(eps2*N*fa)*R2*G1[l2]
       
    print(overlapping_other_l2)
    n_overlap1=sum(-overlapping_other_l1[overlapping_other_l1==-1])
    n_overlap2=sum(-overlapping_other_l2[overlapping_other_l2==-1])
    if (n_overlap1!=n_overlap2):
        print('n_overlap1',n_overlap1)
        print('n_overlap2',n_overlap2)
        print(np.array([overlapping_other_l1,overlapping_other_l2]))
        raise Exception('Non-equal number of overlaps')
        
    print('partial sum',out)
    out=out+N*fa*p2.G0[lam0_2,lam_2]*p1.G0[lam0_1,lam_1]
    out=out+p2.dG0[lam0_2,lam_2]*p1.G0[lam0_1,lam_1]
    out=out+p1.dG0[lam0_1,lam_1]*p2.G0[lam0_2,lam_2]
    print('p1.G0',p1.G0[lam0_1,lam_1])
    print('N term',N*fa*p2.G0[lam0_2,lam_2]*p1.G0[lam0_1,lam_1])
    print('21 term',p2.dG0[lam0_2,lam_2]*p1.G0[lam0_1,lam_1])
    print('12 term',p1.dG0[lam0_1,lam_1]*p2.G0[lam0_2,lam_2])
    print('final sum',out)
    return out

In [55]:
fa=0.5
N=[0.01,0.1,1.0,10,100]
ana=np.zeros((nkpts,len(N)),dtype=type(1+1j))
dub=np.zeros((nkpts,len(N)),dtype=type(1+1j))
for ii in range(0,nkpts):
    lam_1=0
    lam0_1=1
    lam_2=1
    lam0_2=0
    for nn in range(0,len(N)):
        #IAAAexplicit(N,fa,lam0_1,lam_1,lam0_2,lam_2,p1,p2)
        ana[ii,nn]=mp.IAAA(N[nn],fa,\
                                lam0_1,lam_1,\
                                lam0_2,lam_2,\
                                props1[ii],props2[ii])
        dub[ii,nn]=mp.IAAAexplicit(N[nn],fa,\
                                    lam0_1,lam_1,\
                                    lam0_2,lam_2,\
                                    props1[ii],props2[ii])

In [56]:
nn=2
ii=105
print('N',N[nn])
print('K',K[ii])
ana[ii,nn]=IAAA(N[nn],fa,\
                        lam0_1,lam_1,\
                        lam0_2,lam_2,\
                        props1[ii],props2[ii])

N 1.0
K 63.6824994472
alligned l1, 0 l2 0 t1 (-8.58452055182e-08-2.5720729407e-07j)
alligned l1, 0 l2 0 t2 (1.26177201337e-09-2.07191250201e-08j)
alligned l1, 0 l2 0 t3 (1.26177201337e-09-2.07191250201e-08j)
alligned l1, 1 l2 1 t1 (-8.58452055182e-08+2.5720729407e-07j)
alligned l1, 1 l2 1 t2 (1.26177201337e-09+2.07191250201e-08j)
alligned l1, 1 l2 1 t3 (1.26177201337e-09+2.07191250201e-08j)
alligned l1, 2 l2 2 t1 (-1.39680421503e-10+5.74080866088e-11j)
alligned l1, 2 l2 2 t2 (-2.0340306843e-11-2.11779872277e-11j)
alligned l1, 2 l2 2 t3 (-2.0340306843e-11-2.11779872277e-11j)
alligned l1, 3 l2 3 t1 (-1.39680421503e-10-5.74080866088e-11j)
alligned l1, 3 l2 3 t2 (-2.0340306843e-11+2.11779872277e-11j)
alligned l1, 3 l2 3 t3 (-2.0340306843e-11+2.11779872277e-11j)
alligned l1, 4 l2 4 t1 (7.158080134e-14+1.24436802917e-13j)
alligned l1, 4 l2 4 t2 (-1.85549621063e-14+3.74629844115e-14j)
alligned l1, 4 l2 4 t3 (-1.85549621063e-14+3.74629844115e-14j)
alligned l1, 5 l2 5 t1 (7.158080134e-14-1.2443

In [57]:
fig, ax = plt.subplots()
for ii in range(0,len(N)):
    col=float(ii)/len(N)
    ax.loglog(K.real,abs(ana[:,ii].real),\
                color=[col, 0.0, 1-col],label='resum '+str(N[ii]))
    ax.loglog(K.real,abs(dub[:,ii]),':',\
                color=[col, 0.0, 1-col],label='expl '+str(N[ii]))
ax.legend()#bbox_to_anchor=(1.05,1),loc=2)
plotlog(2e4,1e-5,-2)
ax.set_xlabel('K')
ax.set_title('A'+str(lam_1)+',A'+str(lam_2)+',A'+str(lam0_2))
ax.set_ylim([10**-18,10**15])
print(lam_1,',',lam_2,',',lam0_2)
plt.show()

0 , 1 , 0


In [ ]:
nl=5
out=np.zeros((nkpts,len(N)),dtype=type(1+1j))
for ii in range(0,nkpts):
    for l in range(0,(nl)):
        out[ii,l]=props1[ii].res[l][lam0_1,lam_1]

In [ ]:
fig, ax = plt.subplots()
for ii in range(0,nl):
    col=float(ii)/nl
    ax.loglog(K.real,abs(out[:,ii].real),\
                color=[col, 0.0, 1-col],label='resum '+str(N[ii]))
ax.legend(bbox_to_anchor=(1.05,1),loc=2)
ax.set_xlabel('K')
ax.set_title('res')
print('lam',lam_1,'lam0',lam_0)
ax.set_ylim([10**-25,10])
plt.show()

In [ ]:
nl=5
out=np.zeros((nkpts,len(N),nl),dtype=type(1+1j))
for ii in range(0,nkpts):
    for nn in range(0,len(N)):
        for l in range(0,(nl)):
            out[ii,nn,l]=props1[ii].eig[l]

In [ ]:
fig, ax = plt.subplots()
for ii in range(0,nl):
    col=float(ii)/nl
    ax.loglog(K.real,abs(out[:,0,ii].real),\
                color=[col, 0.0, 1-col],label='resum '+str(N[ii]))
ax.legend(bbox_to_anchor=(1.05,1),loc=2)
ax.set_xlabel('K')
ax.set_title('eig')

plt.show()

#### ABB

In [ ]:
N=[0.01,0.1,1.0,10,100]
ana=np.zeros((nkpts,len(N)),dtype=type(1+1j))
dub=np.zeros((nkpts,len(N)),dtype=type(1+1j))
fa=0.4466
for ii in range(0,nkpts):
    lam_1=0
    lam0_1=0
    lam_2=0
    lam0_2=0
    for nn in range(0,len(N)):
        ana[ii,nn]=mp.IABBresum(N[nn],fa,\
                                lam0_1,lam_1,\
                                lam0_2,lam_2,\
                                props1[ii],props2[ii])
        #dub[ii,nn]=IAAAexplicit(N[nn],lam0,lam,props1[ii],props2[ii])

In [ ]:
fig, ax = plt.subplots()
for ii in range(0,len(N)):
    col=float(ii)/len(N)
    ax.loglog(K.real,abs(ana[:,ii]),\
                color=[col, 0.0, 1-col],label='resum '+str(N[ii]))
    #ax.loglog(K.real,abs(dub[:,ii]),'--',\
    #            color=[col, 0.0, 1-col],label='expl '+str(N[ii]))
ax.legend(bbox_to_anchor=(1.05,1),loc=2)
plotlog(2e4,1e-9,-3)
ax.set_xlabel('K')
ax.set_title('ABB')
ax.set_ylim([10**-18,10**15])
plt.show()

## 4 point

In [ ]:
nlam=5
ORDEig=25
d=3
mu1=0
mu2=1
mu3=0

nkpts=100
K=np.logspace(-2,5,nkpts)
props1=[] # list of propagators at each K
props2=[] # list of propagators at each K
props3=[] # list of propagators at each K
for ii in range(0,nkpts):

    name=ii
    props1.append( propagator.propagator(name,1.0*K[ii],mu1,nlam=nlam) )

    name=ii+0.5
    props2.append( propagator.propagator(name,2.0*K[ii],mu2,nlam=nlam) )
    
    name=ii+0.25
    props3.append( propagator.propagator(name,1.0*K[ii],mu3,nlam=nlam) )

In [ ]:
N=[0.001,0.01,0.1,1.0,10,100,1000]
ana=np.zeros((nkpts,len(N)),dtype=type(1+1j))
dub=np.zeros((nkpts,len(N)),dtype=type(1+1j))
fa=0.4466
for ii in range(0,nkpts):
    lam_1=0
    lam0_1=2
    lam_2=2
    lam0_2=1    
    lam_3=1
    lam0_3=0
    
    for nn in range(0,len(N)):
        ana[ii,nn]=mp.IAAAAresum(N[nn],fa,\
                                 lam0_1,lam_1,\
                                 lam0_2,lam_2,\
                                 lam0_3,lam_3,\
                                 props1[ii],props2[ii],props3[ii])
        if np.isnan(ana[ii,nn]):
            print('Nan Found')
        #dub[ii,nn]=mp.IAAAAexplicit(N[nn],fa,\
        #                         lam0_1,lam_1,\
        #                         lam0_2,lam_2,\
        #                         lam0_3,lam_3,\
        #                         props1[ii],props2[ii],props3[ii])

In [ ]:
fig, ax = plt.subplots()
for ii in range(0,len(N)):
    col=float(ii)/len(N)
    ax.loglog(K.real,ana[:,ii].real,'-',\
                color=[col, 0.0, 1-col],label='resum '+str(N[ii]))
    ax.loglog(K.real,-ana[:,ii].real,'--',\
                color=[col, 0.0, 1-col],label='resum '+str(N[ii]))
    #ax.loglog(K.real,-dub[:,ii].real,':',\
    #            color=[col, 0.0, 1-col],label='expl '+str(N[ii]))
#ax.legend(bbox_to_anchor=(1.05,1),loc=2)
#ax.legend(bbox_to_anchor=(1.05,1))
plotlog(2e3,1e-4,-1)
plotlog(2e3,1e-5,-2)
plotlog(2e3,1e-6,-3)
plotlog(2e3,1e-7,-4)
ax.set_xlabel('K')
ax.set_title('AAAA')
ax.set_ylim([10**-18,10**15])
plt.show()